In [1]:
import pandas as pd
import numpy as np

In [2]:
ak_air = pd.read_csv("Alaska Air Quality Data.csv")

In [3]:
ak_air.head()

,location_id,location_name,FIPS,cause_id,cause_name,sex_id,sex,year_id,mx,lower,upper
0,524,Alaska,2,294,All causes,3,Both,1980,1167.420441,1136.460238,1197.922323
1,524,Alaska,2,294,All causes,3,Both,1981,1139.673100,1109.550096,1171.278428
2,524,Alaska,2,294,All causes,3,Both,1982,1106.338700,1078.662094,1133.052351
3,524,Alaska,2,294,All causes,3,Both,1983,1097.128483,1068.838525,1124.720788
4,524,Alaska,2,294,All causes,3,Both,1984,1079.916615,1055.790246,1106.222503


In [4]:
us_mort = pd.read_csv("COUNTY_MORTALITY_RATES_1980-2014_NATIONAL.csv")

In [5]:
us_mort.head()

,HIV/AIDS and tuberculosis,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Location,FIPS,"Mortality Rate, 1980*","Mortality Rate, 1985*","Mortality Rate, 1990*","Mortality Rate, 1995*","Mortality Rate, 2000*","Mortality Rate, 2005*","Mortality Rate, 2010*","Mortality Rate, 2014*","% Change in Mortality Rate, 1980-2014"
1,United States,NaN,"1.52 (1.44, 1.61)","3.16 (3.11, 3.22)","11.45 (11.34, 11.56)","16.61 (16.48, 16.74)","5.97 (5.92, 6.02)","4.87 (4.83, 4.91)","3.20 (3.17, 3.22)","2.66 (2.63, 2.69)","74.35 (64.77, 84.14)"
2,Alabama,1,"1.46 (1.33, 1.59)","2.15 (2.03, 2.27)","8.03 (7.79, 8.25)","14.10 (13.76, 14.43)","5.47 (5.30, 5.65)","4.67 (4.51, 4.83)","3.35 (3.21, 3.50)","2.94 (2.80, 3.08)","101.52 (82.33, 124.12)"
3,"Autauga County, Alabama",1001,"0.95 (0.68, 1.29)","1.44 (1.12, 1.86)","6.57 (5.49, 7.84)","10.75 (9.18, 12.50)","3.62 (2.94, 4.37)","3.32 (2.66, 4.14)","2.37 (1.82, 3.07)","2.30 (1.68, 2.96)","143.15 (65.10, 245.14)"
4,"Baldwin County, Alabama",1003,"0.84 (0.63, 1.12)","1.44 (1.18, 1.75)","6.51 (5.71, 7.40)","11.18 (9.89, 12.59)","3.82 (3.28, 4.44)","3.18 (2.71, 3.72)","2.27 (1.88, 2.72)","2.01 (1.64, 2.46)","138.20 (69.82, 240.56)"


In [ ]:
table = pivot_table(df, values='D', index=['A', 'B'], columns=['C'], aggfunc=np.sum)